# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download(['punkt', 'wordnet', 'stopwords'])

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from sqlalchemy import create_engine
from time import time
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
con = engine.connect()
engine.table_names()

df = pd.read_sql_table("InsertTableName", con)
# Create Train Test datasets 
X = df.filter(items=['id', 'message', 'original', 'genre'])

#Exckuding 'child_alone' as we saw in ETL pipeline notebook
y = df.drop(['message', 'original', 'genre', 'child_alone'], axis=1)

# Changing '2' to '1' in related as we saw in ETL pipeline notebook
y['related']=y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
# Tokenization function
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# ML Pipeline
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data to create Train Test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])

# Testing metrics
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

# Training metrics
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.89      0.86      5009
               request       0.84      0.38      0.52      1164
                 offer       0.00      0.00      0.00        26
           aid_related       0.76      0.40      0.52      2741
          medical_help       0.36      0.01      0.02       502
      medical_products       0.33      0.02      0.03       317
     search_and_rescue       0.00      0.00      0.00       177
              security       0.00      0.00      0.00       125
              military       1.00      0.01      0.02       209
                 water       0.87      0.12      0.22       440
                  food       0.81      0.24      0.37       733
               shelter       0.88      0.10      0.19       555
              clothing       0.40      0.02      0.04        96
                 money       1.00      0.02      0.04       150
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f90cc4f6620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm

In [8]:
# Defining parameters for grid search
parameters = {'clf__max_depth': [10, None],
              'clf__min_samples_leaf': [1],
              'clf__min_samples_split': [3, 6],
              'clf__n_estimators': [10, 20]}

# Grid Search
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  9.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [10, None], 'clf__min_samples_leaf': [1], 'clf__min_samples_split': [3, 6], 'clf__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])

# Testing metrics
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

# Training metrics
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      5009
               request       0.88      0.44      0.59      1164
                 offer       0.00      0.00      0.00        26
           aid_related       0.80      0.50      0.62      2741
          medical_help       0.71      0.02      0.04       502
      medical_products       0.63      0.04      0.07       317
     search_and_rescue       1.00      0.01      0.02       177
              security       0.00      0.00      0.00       125
              military       0.25      0.00      0.01       209
                 water       0.92      0.22      0.35       440
                  food       0.89      0.29      0.44       733
               shelter       0.85      0.12      0.20       555
              clothing       0.67      0.02      0.04        96
                 money       1.00      0.02      0.04       150
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
# Finding best parameters of the model
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 6,
 'clf__n_estimators': 20}

The best decision tree model wor this data would have the parameter shown above in the Output. Let's plug these parameters into the Pipeline and see how it goes

In [11]:
# Define params
parameters = {'clf__max_depth': [None],
              'clf__min_samples_leaf': [1],
              'clf__min_samples_split': [6],
              'clf__n_estimators': [20]}

cv_op = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)
cv_op.fit(X_train['message'], y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [None], 'clf__min_samples_leaf': [1], 'clf__min_samples_split': [6], 'clf__n_estimators': [20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)

In [12]:
y_pred_test = cv_op.predict(X_test['message'])
y_pred_train = cv_op.predict(X_train['message'])

# Testing metrics
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

# Training metrics
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.88      5009
               request       0.86      0.43      0.58      1164
                 offer       0.00      0.00      0.00        26
           aid_related       0.79      0.51      0.62      2741
          medical_help       0.82      0.02      0.04       502
      medical_products       0.44      0.01      0.02       317
     search_and_rescue       1.00      0.02      0.03       177
              security       0.00      0.00      0.00       125
              military       0.00      0.00      0.00       209
                 water       0.92      0.12      0.22       440
                  food       0.86      0.30      0.45       733
               shelter       0.88      0.10      0.17       555
              clothing       0.83      0.05      0.10        96
                 money       1.00      0.02      0.04       150
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [13]:
joblib.dump(pipeline, "classifier.pkl")

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.